In [3]:
# CÉLULA 1 (ATUALIZADA)

# --- FUNÇÕES ---
def add_all_features(df):
    # --- INDICADORES TÉCNICOS ---
    df['sma_20'] = df['close'].rolling(window=20).mean()
    df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
    
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    df['bollinger_upper'] = df['sma_20'] + (df['close'].rolling(window=20).std() * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['close'].rolling(window=20).std() * 2)
    
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema_12 - ema_26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    
    # --- NOVA FEATURE: ATR (Average True Range) ---
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    df['atr_14'] = true_range.rolling(14).mean()
    
    # --- FEATURE DE RETORNO ---
    df['daily_return'] = df['close'].pct_change()
    
    # --- FEATURES DE TEMPO ---
    df['day_of_week'] = df.index.dayofweek.astype(float)
    df['day_of_month'] = df.index.day.astype(float)
    df['week_of_year'] = df.index.isocalendar().week.astype(float)
    df['month'] = df.index.month.astype(float)
    
    # Remover linhas com NaN resultantes dos cálculos de indicadores
    return df.dropna()

# O resto das funções (create_labels, create_sequences) permanece igual.
# ...

print("Função 'add_all_features' atualizada com ATR.")


Função 'add_all_features' atualizada com ATR.


In [2]:
# CÉLULA 2: Loop de Otimização para o Timeframe de 15m

# --- PARÂMETROS GERAIS ---
LOOK_FORWARD = 48      # Olhar 12 horas para a frente
TIME_STEPS = 120       # Usar as últimas 30 horas de dados
CONFIDENCE = 0.75      # Limiar de confiança

# --- PARÂMETROS PARA OTIMIZAR ---
profit_targets = [0.005, 0.01, 0.015]  # Alvos de 0.5%, 1.0%, 1.5%
stop_losses = [0.0025, 0.005, 0.0075] # Stops de 0.25%, 0.5%, 0.75%

# Lista para guardar os resultados
results = []

# Carregar os dados uma única vez
btc_df_raw = pd.read_feather('../user_data/data/binance/BTC_USDT-15m.feather')
btc_df_raw['date'] = pd.to_datetime(btc_df_raw['date'])
btc_df_raw.set_index('date', inplace=True)

# --- INÍCIO DO LOOP ---
for pt in profit_targets:
    for sl in stop_losses:
        print(f"\\n--- INICIANDO TESTE COM: Alvo={pt*100:.2f}%, Stop={sl*100:.2f}% ---")
        
        # 1. PREPARAÇÃO DOS DADOS (dentro do loop)
        df_labeled = create_labels(btc_df_raw.copy(), LOOK_FORWARD, pt, sl)
        df_featured = add_all_features(df_labeled)
        
        y = df_featured['label']
        X = df_featured.drop(columns=['label', 'open', 'high', 'low', 'close', 'future_price', 'price_change'])
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train.values, TIME_STEPS)
        X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test.values, TIME_STEPS)
        
        y_train_adj = y_train_seq + 1

        # 2. TREINO DO MODELO (dentro do loop)
        model = Sequential([
            LSTM(units=128, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
            Dropout(0.3),
            LSTM(units=64, return_sequences=True),
            Dropout(0.3),
            LSTM(units=32, return_sequences=False),
            Dropout(0.3),
            Dense(units=3, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        model.fit(X_train_seq, y_train_adj, epochs=50, batch_size=64, validation_split=0.1, callbacks=[early_stopping], verbose=0) # verbose=0 para um output mais limpo

        # 3. BACKTESTING (dentro do loop)
        predictions_test = model.predict(X_test_seq)
        
        start_index = len(X_train) + TIME_STEPS
        test_prices = df_featured['close'].iloc[start_index:]
        backtest_df = pd.DataFrame({'market_return': test_prices.pct_change()})
        
        backtest_df['signal'] = 0
        backtest_df.loc[predictions_test[:, 2] > CONFIDENCE, 'signal'] = 1
        backtest_df.loc[predictions_test[:, 0] > CONFIDENCE, 'signal'] = -1
        
        backtest_df['strategy_return'] = backtest_df['market_return'] * backtest_df['signal'].shift(1)
        
        cumulative_return = (1 + backtest_df['strategy_return']).prod() - 1
        
        # Guardar o resultado
        results.append({
            'profit_target': pt,
            'stop_loss': sl,
            'strategy_return': cumulative_return * 100
        })
        print(f"  Resultado da Estratégia: {cumulative_return * 100:.2f}%")

# --- FIM DO LOOP ---

# 4. RESULTADOS FINAIS
print("\\n--- TABELA DE RESULTADOS DA OTIMIZAÇÃO (15m) ---")
results_df = pd.DataFrame(results)
display(results_df.sort_values(by='strategy_return', ascending=False))


\n--- INICIANDO TESTE COM: Alvo=0.50%, Stop=0.25% ---


2025-09-20 17:11:55.411821: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-09-20 17:11:55.413863: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-20 17:11:55.415184: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-20 17:11:55.416536: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-20 17:11:55.418389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When u

1091/1091 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step
  Resultado da Estratégia: 0.00%
\n--- INICIANDO TESTE COM: Alvo=0.50%, Stop=0.50% ---


/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1091/1091 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step
  Resultado da Estratégia: 0.00%
\n--- INICIANDO TESTE COM: Alvo=0.50%, Stop=0.75% ---


...

/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1091/1091 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step
  Resultado da Estratégia: 0.00%
\n--- INICIANDO TESTE COM: Alvo=1.50%, Stop=0.75% ---


/Users/marianobernini/miniconda/envs/botenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1091/1091 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step
  Resultado da Estratégia: 0.00%
\n--- TABELA DE RESULTADOS DA OTIMIZAÇÃO (15m) ---


,profit_target,stop_loss,strategy_return
0,0.005,0.0025,0.0
1,0.005,0.0050,0.0
2,0.005,0.0075,0.0
3,0.010,0.0025,0.0
4,0.010,0.0050,0.0
5,0.010,0.0075,0.0
6,0.015,0.0025,0.0
7,0.015,0.0050,0.0
8,0.015,0.0075,0.0
